<a href="https://colab.research.google.com/github/Amirnaderiy/Telegram-bot/blob/main/Telegrm_NewsBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install python-telegram-bot==13.7 requests beautifulsoup4 transformers torch nest-asyncio


In [12]:
import os
import asyncio
import aiohttp
from telegram import Bot, ParseMode
from telegram.error import TelegramError
from bs4 import BeautifulSoup
import logging
from transformers import pipeline
import nest_asyncio
import random

# Enable nested event loops (required for Colab)
nest_asyncio.apply()

# Configure logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize the bot with your token
TOKEN = "6186938893:AAG2DC375_esINgx2LaHCTGxk8a1F67BlLs"
CHANNEL_ID = "@archive_tvseries_persian_tvbot"  # Replace with your channel username or ID

# Initialize the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Store previously posted news to avoid duplicates
posted_news = set()

async def fetch_news(session, url, source_name, article_selector, title_selector, link_selector, description_selector):
    """Generic function to fetch news from various sources."""
    try:
        async with session.get(url) as response:
            if response.status == 200:
                content = await response.text()
                soup = BeautifulSoup(content, 'html.parser')
                articles = soup.select(article_selector)

                news_items = []
                for article in articles[:5]:  # Limit to 5 news items
                    try:
                        title_element = article.select_one(title_selector)
                        link_element = article.select_one(link_selector)
                        description_element = article.select_one(description_selector)

                        if title_element and link_element and description_element:
                            title = title_element.text.strip()
                            link = link_element['href']
                            if not link.startswith('http'):
                                link = url + link
                            description = description_element.text.strip()
                            news_items.append({'title': title, 'link': link, 'description': description, 'source': source_name})
                        else:
                            logger.warning(f"Skipping an article from {source_name} due to missing elements")
                    except Exception as e:
                        logger.warning(f"Error processing an article from {source_name}: {str(e)}")

                return news_items
            else:
                logger.warning(f"Failed to fetch news from {source_name}. Status code: {response.status}")
                return []
    except Exception as e:
        logger.error(f"Error fetching news from {source_name}: {str(e)}")
        return []

async def get_tv_series_news(session):
    """Fetch TV series news from multiple sources."""
    news_sources = [
        ("https://www.tvguide.com/news/", "TV Guide", 'article.article-news', 'h3.article-title', 'a', 'p.article-description'),
        ("https://deadline.com/v/tv/", "Deadline", 'article', 'h2.pmc-latest-news__headline', 'a', 'div.pmc-latest-news__description'),
        ("https://collider.com/tv/", "Collider", 'article.article-card', 'h3.cs-title', 'a.cs-link', 'p.cs-excerpt'),
        ("https://tvline.com/", "TVLine", 'article.entry-box', 'h2.entry-title', 'a', 'div.entry-content'),
        ("https://variety.com/v/tv/", "Variety", 'div.c-title-list__item', 'h3', 'a', 'div.c-title-list__description'),
        ("https://www.hollywoodreporter.com/c/tv/", "The Hollywood Reporter", 'div.card', 'h3.card__title', 'a.card__link', 'div.card__description')
    ]

    all_news = []
    for source in news_sources:
        news_items = await fetch_news(session, *source)
        all_news.extend(news_items)
        await asyncio.sleep(random.uniform(1, 3))  # Random delay between requests

    return all_news

def summarize_text(text, max_length=150):
    """Summarize the given text using the BART model."""
    summary = summarizer(text, max_length=max_length, min_length=50, do_sample=False)
    return summary[0]['summary_text']

async def format_message(news_item):
    """Format the news item into a visually appealing message."""
    title = f"🎬 <b>{news_item['title']}</b>"
    summary = summarize_text(news_item['description'])
    source = f"Source: {news_item['source']}"
    message = f"{title}\n\n{summary}\n\n{source}\n\n<a href='{news_item['link']}'>Read more</a>"
    return message

async def send_message_to_channel(bot, message):
    """Send a formatted message to the channel."""
    try:
        await bot.send_message(chat_id=CHANNEL_ID, text=message, parse_mode=ParseMode.HTML, disable_web_page_preview=True)
        logger.info(f"Message sent successfully: {message[:50]}...")
    except TelegramError as e:
        logger.error(f"Error sending message: {e}")

async def main():
    bot = Bot(TOKEN)

    async with aiohttp.ClientSession() as session:
        while True:
            try:
                news_items = await get_tv_series_news(session)
                for item in news_items:
                    news_id = f"{item['source']}_{item['title']}"
                    if news_id not in posted_news:
                        formatted_message = await format_message(item)
                        await send_message_to_channel(bot, formatted_message)
                        posted_news.add(news_id)
                        await asyncio.sleep(random.uniform(1, 2))  # Random delay between posts

                # Wait for a short time before fetching news again
                await asyncio.sleep(random.uniform(5, 10))
            except Exception as e:
                logger.error(f"An error occurred in the main loop: {e}")
                await asyncio.sleep(30)  # Wait for 30 seconds before retrying

# Function to start the bot
def run_bot():
    asyncio.run(main())

print("Bot is ready to run. Use the run_bot() function to start it.")
print("The bot will run continuously until stopped.")

Bot is ready to run. Use the run_bot() function to start it.
The bot will run continuously until stopped.


In [ ]:
run_bot()